
<p style="text-align:right;"><img src="https://upload.wikimedia.org/wikipedia/ar/c/c3/Umm_Al-Qura_University_logo.png" alt="Umm al-Qura Univeisty" width="100" height="100"> </img></p>

# <p style="text-align:center;"><b>A Hybrid-Agent Implementation of the Wumpus World</b></p>

by Ahmad Hasanain, College of Engineering at al-Lith

Refrences:

1. AIMA: Stuart Russell & Peter Norvig, Artificial Intelligence: A Modern Approach, 4th ed. (2020,
Global), Page 259
https://www.amazon.sa/Artificial-Intelligence-Modern-Approach-Global/dp/1292401133

2. Inference Function
https://docs.sympy.org/latest/modules/logic.html

3. DPLL Algorithm
https://en.wikipedia.org/wiki/DPLL_algorithm





In [ ]:
import MP2Game
import time
from sympy import ask, Symbol, Equivalent
from sympy import satisfiable, And, Or
cnts = MP2Game.HuntTheWumpusGame(human = False)

# --------------- initialization code ---------------


#  

# Define variables 

# 1. define the propositional symbols in the AI problem
G = dict()
for k in ["P","B","W","S","R"]:
    G.update( { k : dict() } )
    for i in range(4):
        G[k].update( { i : dict() } )
        for j in range(4):
            G[k][i].update( { j : Symbol( f"{k}[{i},{j}]" ) } )

# 2. define a knowledge-base set
KB = set() # this is for the rules of the game

# 3. resetable variables 
KK = set() # this is another KB but for the observations
move_history = [] # this is for finding the shooting location by backtracking the agents steps 
Wumpus_alive = True



# Add the rules of the game to the knowledge base (KB)

# 1. there is no Pit nor Wumpus in 0,0
KB.add( ~G['P'][0][0] ) 
KB.add( ~G['W'][0][0] )

# for each sqaure
for i in range(4):
    for j in range(4):
        
        # 2. The square is Risky if there is a Pit or Wumpus in it
        KB.add( Equivalent( G['R'][i][j] , G['P'][i][j] | G['W'][i][j] ) )
        
        # find legally adjacent squares for both Pit and Wumpus of the current square [i, j]
        AP = set()
        AW = set()
        for a in range( max(i-1,0) , min(i+2,4) ):
            AP.add( G['P'][a][j] )
            AW.add( G['W'][a][j] )
        for b in range( max(j-1,0) , min(j+2,4) ):
            AP.add( G['P'][i][b] )
            AW.add( G['W'][i][b] )

        # 3. the square is Beezy if there is an adjacnet Pit
        KB.add( Equivalent( G['B'][i][j] , Or(*AP) )  ) # B[i,j] <=> P[i-1,j] | P[i+1,j] | P[i,j-1] | P[i,j+1] 
               
        # 4. the square is Stenchy if there is an adjacnet Wumpus
        KB.add( Equivalent( G['S'][i][j] , Or(*AW) )  ) # S[i,j] <=> W[i-1,j] | W[i+1,j] | W[i,j-1] | W[i,j+1] 



def eliminateLocatableWumpus():
    global G, KB, KK, cnts, move_history
    # for each square, check if a Wumpus is locatable in it.
    for i in range(4):
        for j in range(4):
            # if the agent finds a Wumpus
            if not satisfiable( And( ~G['W'][i][j] , *KB, *KK ) , algorithm = "dpll2" ) :
                # backtrack the history until the agent is in the same column or row as the Wumpus
                move_history.pop() # remove current location from history
                while cnts.x != i and cnts.y != j :
                    k = move_history.pop()
                    cnts.go_to( *k )
                # then shoot at the Wumpus
                cnts.shoot_at( i, j )

                # remove the wumpus from the observation knowledge base
                KK.add( ~G['W'][i][j] )
                # remove the sorrounding stench from the observation knowledge base
                for a in range( max(i-1,0) , min(i+2,4) ):
                    for b in range( max(j-1,0) , min(j+2,4) ):
                        KK.discard( G['S'][a][b] )
                        KK.add( ~G['S'][a][b] )
                return True # return True for shot
    return False # return False if not found

while True: 
    sensors = cnts.percept()
    if not sensors:
        # --------------- reset code ---------------
        KK = set()
        move_history = []
        Wumpus_alive = True
        cnts.start()
        continue
    # here is the location of the agent at this time step for simplification
    x = cnts.x
    y = cnts.y
    
    # --------------- update agent's knowledge base: the agent is still alive, so there is no Wumpus or Pit in the current square
    move_history.append( [x,y] )
    KK.add( ~G['W'][x][y] ) # the agent is still alive, so there is no Wumpus in the current square
    KK.add( ~G['P'][x][y] ) # 
    
    # did the agent find Glitter in the current square?
    if sensors[2] == "Glitter":
        cnts.action( "Grab" ) # then Grab it
        
    # did the agent hear Scream in the last sensor reading
    if sensors[4] == "Scream":
        Wumpus_alive = False # then there is no Wumpus
        # print( "Scream Heard" )
    
    # if Wumpus is alive, did the agent find Stench in the current square?
    if Wumpus_alive & (sensors[0] == "Stench") :
        KK.add( G['S'][x][y] ) # there is a Stench in the current square
    else:
        KK.add( ~G['S'][x][y] ) # there is no Stench in the current square
        
    # did the agent find Breeze in the current square?
    if sensors[1] == "Breeze":
        KK.add( G['B'][x][y] ) # there is a Breeze in the current square
    else:
        KK.add( ~G['B'][x][y] ) # there is no Breeze in the current square
    
    # if wumpus is alive and is shot
    if Wumpus_alive & eliminateLocatableWumpus():
        continue # continue to the next loop to check the sensor
    
    # Find the next safe move
    frontier = cnts.frontier() # this finds the next candidate moves
    frontier.sort(key=lambda k: abs(k[0]-x) + abs(k[1]-y)  ) # this orders the candidate moves by their distance from the agent
    destination = [-1,0] # return to [0,0] and Climb if no safe square is found to move to.
    # for each coordinate pair in the frontier  
    for d in frontier:
        # --------------- if the null hypothesis is unsatisfiable ---------------
        if not satisfiable( And( G['R'][d[0]][d[1]] , *KB, *KK ) , algorithm = "dpll2" ) :                                
            destination = d # update the destination with the safe move
            break # and stop the search
            
    # go to the next destination or stop the system's loop when the window is closed
    response = cnts.action( destination ) 
    if not response:
        break
    # time.sleep(0.2) # else, continue the system's main loop
        
        

        
        

### You can test the game yourself by running the following cell

In [ ]:
import MP2Game
_ = MP2Game.HuntTheWumpusGame()